# Homework 3: Star formation in galaxies

The homework should be submitted as a python notebook. Make all the plots in the python notebook.

Please email the homework to hdenes@yachaytech.edu.ec by the <b>end of the day (midnight) by the 15th of December.</b>

Total points: 20

Please let me know if you have any questions. 

<b>Important notes: </b> 
- The tutorial notebooks contain the methods that you can use to solve the homework. 
- If you have problems with some of the specific astrophysics packages, make sure to check the versions. Not all versions of the packages are compatible with each other. 

## (20 points) Analysing star formation properties of galaxies 

First dwonload the relevant data from the Sloan DIgital Sky Survey (SDSS) data base. The code for this is provided. In the cell below.  

- The SpecObj table contains basic information about the observation and the object, such as the redshift. 
- The galaxy table provides useful photometric measurements for colors and images. 
- The three GalSpec table each give products derived from spectroscopy. 

The H$\alpha$ emission line is a convienient indicator of the star formation rate. 

We want galaxies with quality measurements of this line strength, so a S/N restriction is specified. We also want the H$\beta$ emission line, because as we will see below, this will assist us in accounting for dust absorption along our line of sight.

In [ ]:
# Import relevant libraries

import pandas as pd # data analysis 
import numpy as np # more data analysis

from matplotlib import pyplot as plt # another plotting library
from matplotlib.image import imread

from astropy.constants import c # useful constants
from astropy import units as u # useful for unit conversions
from astropy import coordinates as coords
from astropy.coordinates import SkyCoord
from astropy.io import fits
from astropy.io import ascii

from astroquery.sdss import SDSS
from astroquery.simbad import Simbad

# Python standard-libraries to download data from the web
from urllib.parse import urlencode
from urllib.request import urlretrieve

#only here to display images
from IPython.display import Image

In [ ]:
# Create a query to fetch star forming galaxies

query = 'SELECT TOP 10000 s.plate, s.fiberid, s.mjd, s.ra, s.dec, s.z, s.zwarning, '
query += 'g.h_beta_flux, g.h_alpha_flux, '
query += 'g.h_beta_flux_err, g.h_alpha_flux_err, '
query += 'e.bptclass, e.lgm_tot_p50, '
query += 'i.lick_hd_a_sub, i.d4000_n_sub, '
query += 'i.lick_hd_a_sub_err, i.d4000_n_sub_err, '
query += 'p.petror90_r '
query += 'FROM GalSpecLine AS g, SpecObj AS s, GalSpecExtra AS e, GalSpecIndx AS i, galaxy AS p ' 
query += 'WHERE '
query += 's.specobjid = g.specobjid '
query += 'AND s.specobjid = e.specobjid '
query += 'AND s.specobjid = i.specobjid '
query += 'AND s.bestobjid = p.objid '
query += 'AND bptclass = 1 '
query += 'AND s.z BETWEEN 0.04 and 0.1 '#-- Lower limit needed for global parameters w/ small aperture
query += 'AND h_alpha_flux > h_alpha_flux_err*5.0 ' #-- Make sure Balmer lines have adequate S/N
query += 'AND h_beta_flux > h_beta_flux_err*5.0 '
query += 'AND d4000_n_sub > d4000_n_sub_err*5.0 '
query += 'AND h_alpha_flux_err > 0 ' #-- Consistency check
query += 'AND h_beta_flux_err > 0 '
query += 'AND h_alpha_flux > 2.7*h_beta_flux ' #-- Ensures proper continuum subtraction
query += 'AND lgm_tot_p50 > 0 '
query += 'AND sigma_balmer < 509.55 ' #-- Exclude the broad line emitting galaxies
query += 'AND s.class = \'GALAXY\' '  #-- Looking for galaxies, not stars
query += 'AND s.zwarning = 0 '
#print (query) # useful for debugging, remove first # to uncomment

# send query to SDSS
gals = SDSS.query_sql(query,data_release=16)

gals

### Dereddening the Spectrum

In order to determine the star formation rate (SFR) in each galaxy, we need to get the H $\alpha$ luminosity L(H$\alpha$) emitted from each galaxy. The SQL query gives us the observed flux F(H$\alpha$), but this doesn’t account for dust absorbing some of this light as it travels from a galaxy to the point of observation. Fortunately, we can correct for this “extinction” by making use of the fact that dust preferentially absorbs shorter wavelength light to “deredden” the spectrum. In ideal conditions without dust, we can assume that 
F(H$\alpha$)/F(H$\beta$) = 2.86 in star forming regions. Let’s deredden the H$\alpha$ flux.

In [ ]:
# Deredden emission lines according to Calzetii+ 2000
# wavelength supplied in angstroms

def deredden(lum,wl,Hb,Ha):
    if (wl >= 6300 and wl <= 22000):
        wl = wl/1e4 # convert to microns
        kwl = 2.659*(-1.857+1.040/wl)+4.05
    elif (wl >= 1200 and wl < 6300):
        wl = wl/1e4 # convert to microns
        kwl = 2.659*(-2.156+1.509/wl-0.198/wl**2+0.011/wl**3)+4.05
    else:
        print ('Wavelength of range')
    
    # Using k-values calc as above
    kHa = 3.326
    kHb = 4.598
    
    ebmv = 2.5/(kHb-kHa)*np.log10(Ha/Hb/2.86) # 2.86 for SF; 3.1 for AGN
    lum_dr = lum*10**(0.4*kwl*ebmv)
    
    return lum_dr

####

Halpha_flux = gals['h_alpha_flux']
Hbeta_flux = gals['h_beta_flux']

Halpha_flux_dr = deredden(Halpha_flux,6563,Hbeta_flux,Halpha_flux)

According to the GalSpecLine table, the line flux is returned as 10$^{-17}$ erg/s/cm$^2$, so we first need to multiply our dereddened F(H) by 10$^{-17}$ to get actual flux value. Luminosity has units of energy / second, and an erg is a unit of energy. 

1) Calcualte the distance to the galaxies in the sample in cm. Make sure to clarify which hubble constant you are using. 

2) Convert the H$\alpha$ flux into H$\alpha$ luminosity.

Hint, the relavant relations are covered in the lecture slides. Astropy also has functions that are very helpful for calculating distances. 

We can now use the L(H$\alpha$) to determine the SFR in units of M$_{\odot}$/yr using the conversion from Kennicutt+ 1998.

$$\frac{SFR}{M_{\odot} yr^{-1}} = \frac{L(H\alpha)}{1.26 \times 10^{41} ergs \ s^{-1}}$$

3) Calcualte the star formation rate as $log_{10}(SFR)$. Make a histogram of $log_{10}(SFR)$. How is the distribution of the star formation rate?  

### The Star Forming Main Sequence

A common measure of galaxy mass is its stellar mass M$_{\star}$ in units of M$_{\odot}$, however a galaxy’s mass can be defined in many ways. We can roughly envision that M$_{\star}$ is proxy for galaxy size since larger galaxies typically contain more stars.

4) Make a plot of $log_{10}(SFR)$ vs $log_{10}(M_{\star})$. We have $log_{10}(M_{\star})$ provided in the SDSS data as: gals['lgm_tot_p50']. Plot $log_{10}(M_{\star})$ on the x-axis.

Based on the plot, how does the SFR correlate with M$_{\star}$?

The expected correlation is the following: 

$$log_{10}(SFR) = log_{10}M_{\star} \times 1.292 - 12.31$$

5) Make the same plot $log_{10}(SFR)$ vs $log_{10}(M_{\star})$ adding the expected correlation. What can you conclude form this plot? Are there many galaxies that are above the correlation?  

6) What kind of galaxies are the ones with high star formation rates? Are these spirals or ellipticals? Do these galaxies look blue or red? To answere this, download a sample of images for 16 galaxies for which $log_{10}(SFR)$ is between -0.05 and 0.05. You can use the code below to get the image cutouts. 

In [ ]:
#
# Function to display images
#
def get_images(gal_array,description):
    # set thumbnail parameters
    width=200           # image width
    height=200          # height
    pixelsize=0.396     # image scale
    plt.figure(figsize=(15, 15))   # display in a 4x4 grid
    subPlotNum = 1

    i = 0
    new_gals = gal_array
    nGalaxies = len(new_gals)
    for index in range(0,nGalaxies):           # iterate through rows in the DataFrame
        i = i + 1
        if i<17:
            print('Getting image of '+description+str(i)+' of '+str(nGalaxies)+'...')
            if (i == nGalaxies):
                print('Plotting images...')
            scale=2.0*new_gals['petror90_r']/pixelsize/width
            
            pos = coords.SkyCoord(new_gals['ra'][index], new_gals['dec'][index], unit=u.deg, frame='icrs')
            
            cutoutbaseurl = 'http://skyservice.pha.jhu.edu/DR12/ImgCutout/getjpeg.aspx'
            query_string = urlencode(dict(ra=new_gals['ra'][index], dec=new_gals['dec'][index],width=width, height=height, scale=scale[i]))
            url = cutoutbaseurl + '?' + query_string

            # this downloads the image
            image_name = 'cutout_'+str(index)+'.jpg'
            urlretrieve(url, image_name)

            # Display the image
            img = imread(image_name)

            plt.subplot(4,4,subPlotNum)
            subPlotNum += 1
            plt.imshow(img)                               # show images in grid
            plt.title(index)                              # show the object identifier (objId) above the image.

######


7)  What kind of galaxies are the ones with low star formation rates? Are these spirals or ellipticals? Do these galaxies look blue or red? To answere this, download a sample of images for 16 galaxies for which $log_{10}(SFR)$ is between -1.55 and -1.45. You can use the code below to get the image cutouts. 


Galaxies can deviate from the Main Sequence (the line that we plotted) on the SFR-M$_{\star}$ diagram. At a given M$_{\star}$, <b>starburst galaxies have 10x the SFR of the Main Sequence galaxies,</b> while <b>quiescent galaxies have a 0.01x the SFR of the Main Sequence galaxies</b>. 

8) Use these definitions to add two more lines to the $log_{10}(SFR)$ vs $log_{10}(M_{\star})$ plot denoting the location of starbursts and quiescent galaxies.

### The specific star formation rate

It becomes evident from this plot that when we want to identify a highly star forming galaxy, the metric we’re usually after is the specific star formation rate sSFR. It is defined as

$$sSFR = \frac{SFR}{M_{\star}}$$ 

and represents ratio of current star formation to past star formation, or the growth rate of stellar mass in a galaxy. We can add the sSFR to our dataframe to explore the metric in detail.

9) Calcualte the $log_{10}(sSFR)$ and make a histogram of $log_{10}(sSFR)$. Does this histogram look similar to the $log_{10}(SFR)$ histogram? 

10) Get images of galaxies with high sSFR ($log_{10}$(sSFR) > -10) and low sSFR ($log_{10}$(sSFR) < 11.5). These are starburst galaxies and quiescent galaxies. Do these galaxies look the same as the high and low SFR galaxies? Do you see any differences? 

Note: If for some reason there are no galaxies in the given $log_{10}$(sSFR) ranges, then select galaxies from the two ends of the distribution. 